# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# for loading data
import pandas as pd
import sqlite3
from sqlalchemy import create_engine

# for tokenizing
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

# for pipeline build
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

# for pipeline train
from sklearn.model_selection import train_test_split

# for model test
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# for gridcv
import numpy as np
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
def load_data():
    engine = create_engine('sqlite:///database_etl_rhn_udacity.db')
    df = pd.read_sql("SELECT * FROM table_etl_rhn_udacity", engine)
    X = df.message.values
    y = df.iloc[:, 5:].values
    categories = list(df.columns)[5:]
    return X,y,categories

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):

    # tokenize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    tokens = word_tokenize(text)
        
    tokens = [t for t in tokens if t not in stopwords.words("english")] 
                 
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer() 

    # iterate through each token
    clean_tokens = []
    for tok in tokens:        
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# Split data into train and test sets
X, y, categories = load_data()
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [6]:
# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [7]:
# predict on test data
y_pred = pipeline.predict(X_test)
y_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
print(classification_report(y_test,y_pred,target_names=categories))

                        precision    recall  f1-score   support

               related       0.85      0.92      0.88      5043
               request       0.75      0.43      0.55      1123
                 offer       0.00      0.00      0.00        26
           aid_related       0.76      0.60      0.67      2724
          medical_help       0.56      0.06      0.11       489
      medical_products       0.73      0.11      0.19       327
     search_and_rescue       0.71      0.06      0.11       170
              security       0.00      0.00      0.00       110
              military       0.59      0.05      0.10       248
           child_alone       0.00      0.00      0.00         0
                 water       0.81      0.31      0.45       404
                  food       0.83      0.36      0.50       696
               shelter       0.81      0.33      0.47       567
              clothing       0.75      0.12      0.21        97
                 money       0.56      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipeline.get_params()

In [9]:
parameters = {
    'clf__estimator__n_estimators': [50, 200],
    'clf__estimator__min_samples_split': [2, 4]
}

In [10]:
# reduce the optimization time:
# instantiate gridsearchcv having cv=2 (2 fold validation, by default it is set to 5)
# and verbose = 3 to get the detailed realtime progress of the operation.
cv = GridSearchCV(pipeline, param_grid=parameters, cv = 2, verbose = 3)

In [11]:
def display_results(cv, y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test.argmax(axis=1),y_pred.argmax(axis=1))
    
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)
    print("\nBest Parameters:", cv.best_params_)

def main():
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    cv.fit(X_train, y_train)
    y_pred = cv.predict(X_test)

    display_results(cv, y_test, y_pred)

main()

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, score=0.257349201505442, total= 3.4min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.4min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, score=0.25867154918116164, total= 3.5min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  8.7min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200, score=0.2572474824534635, total= 9.4min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200, score=0.26324890652019123, total= 9.5min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=50 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=50, score=0.24778761061946902, total= 3.1min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=50 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=50, score=0.2545010680500458, total= 3.1min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=200 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=200, score=0.2524666870104771, total= 8.1min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=200 
[CV]  clf__estimator__min_samples_spl

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed: 57.8min finished


Labels: [0 1]
Confusion Matrix:
 [[6543    2    4    1    3    1]
 [   0    0    0    0    0    0]
 [   0    0    0    0    0    0]
 [   0    0    0    0    0    0]
 [   0    0    0    0    0    0]
 [   0    0    0    0    0    0]]
Accuracy: 0.949432916285

Best Parameters: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 200}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.